In [7]:
from getpass import getpass
from huggingface_hub import whoami
import os, stat

# a) Ask for your token
token = getpass("🔑 Enter your Hugging Face access token: ")

# b) Fetch your HF username
user = whoami(token=token)["name"]
print(f"Using HF user: {user}")

# c) Write ~/.netrc
netrc_path = os.path.expanduser("~/.netrc")
with open(netrc_path, "w") as f:
    f.write(f"machine huggingface.co\nlogin {user}\npassword {token}\n")
# secure it
os.chmod(netrc_path, stat.S_IRUSR | stat.S_IWUSR)
print(f"Wrote credentials to {netrc_path}")

🔑 Enter your Hugging Face access token:  ········


Using HF user: timnirmal
Wrote credentials to /root/.netrc


In [8]:
!git clone https://huggingface.co/stabilityai/stable-diffusion-3-medium-diffusers sd3_local

Cloning into 'sd3_local'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93 (from 1)
Unpacking objects: 100% (93/93), 1.37 MiB | 1.97 MiB/s, done.


In [18]:
!apt-get update
!apt-get install -y git-lfs

Get:1 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1804 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]   
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3420 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [56.4 kB]
Ge

In [20]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [21]:
# Go to your cloned repo and pull large files
%cd sd3_local
!git lfs pull
%cd .

/workspace/sd3_local


/usr/local/lib/python3.11/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspace/sd3_localcts: 100% (8/8), 16 GB | 224 MB/s                           


In [10]:
import json, os

# Path to the SD3 local clone you made earlier
cfg_path = os.path.join("sd3_local", "transformer", "config.json")

# 1) Load
cfg = json.load(open(cfg_path))

# 2) Remove the problematic key
if "num_attention_heads" in cfg:
    cfg.pop("num_attention_heads")
    print("Dropped num_attention_heads from config")

# 3) Overwrite the file
with open(cfg_path, "w") as f:
    json.dump(cfg, f, indent=2)

print("✅ Patched config.json")


Dropped num_attention_heads from config
✅ Patched config.json


In [26]:
%cd ..

/workspace


In [33]:
!pip uninstall -y diffusers
!pip install git+https://github.com/huggingface/diffusers.git@main

Found existing installation: diffusers 0.34.0
Uninstalling diffusers-0.34.0:
  Successfully uninstalled diffusers-0.34.0
  Cloning https://github.com/huggingface/diffusers.git (to revision main) to /tmp/pip-req-build-1o6chg41
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-1o6chg41
  Resolved https://github.com/huggingface/diffusers.git to commit 3c8b67b3711b668a6e7867e08b54280e51454eb5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.35.0.dev0-py3-none-any.whl size=3941241 sha256=cf94590ddcbcc6a5db64db08e6f9577743bc5d26071bc3bf2c2f23dfdffd779a
  Stored in directory: /tmp/pip-ephem-wheel-cache-o6mg0o36/wheels/a1/ef/6a/3ca5a827bc9370215ec33c7a91527ca9415102c7fff40d00be
Successfully built diffusers


In [39]:
# import os, torch
# from diffusers import UNet2DConditionModel

# # adjust path if needed
# transformer_dir = "sd3_local/transformer"

# # sanity check
# print("Contents:", os.listdir())

# from diffusers import UNet2DConditionModel

# unet = UNet2DConditionModel.from_pretrained(
#     "sd3_local",                # your locally-cloned SD3 repo
#     subfolder="transformer",    # where config.json + safetensors live
#     torch_dtype=torch.float16,
#     device_map={"": "cuda"},          # dispatch modules to GPU(s)
#     low_cpu_mem_usage=False,    # ← disable meta-tensor “low mem” mode
#     # optional: spill less-used weights back to CPU
#     offload_folder="offload",          
#     offload_state_dict_device="cpu",
#     local_files_only=True
# )

# # no .to("cuda") call needed—device_map already placed it on CUDA
# print("Loaded on", next(unet.parameters()).device)

# print("✅ Loaded UNet:", unet)


Contents: ['Untitled.ipynb', 'sd3_local', 'test.ipynb', '.git', 'lora_dataset.json', 'lora.ipynb', 'dataset', 'dataset.zip', 'start.ipynb', '.ipynb_checkpoints']


ValueError: You cannot set `low_cpu_mem_usage` to `False` while using device_map={'': 'cuda'} for loading and dispatching. Please make sure to set `low_cpu_mem_usage=True`.

In [1]:
from diffusers import UNet2DConditionModel
import torch

# 1) Load the full UNet into CPU (real tensors, not meta)
unet = UNet2DConditionModel.from_pretrained(
    "sd3_local",               # your local clone
    subfolder="transformer",   # where config.json + weights live
    torch_dtype=torch.float16,
    low_cpu_mem_usage=False,   # load the actual weights straight into RAM
    device_map=None,           # do NOT auto‐dispatch; load on CPU
    local_files_only=True
)

# 2) Now move all of it to your 4090
unet = unet.to("cuda")

print("✅ UNet is now on", next(unet.parameters()).device)


Some weights of the model checkpoint at sd3_local were not used when initializing UNet2DConditionModel: 
 ['transformer_blocks.21.ff_context.net.0.proj.bias, transformer_blocks.0.attn.to_q.bias, transformer_blocks.3.attn.to_v.bias, transformer_blocks.3.attn.add_k_proj.bias, transformer_blocks.23.attn.add_k_proj.weight, transformer_blocks.22.ff.net.2.bias, transformer_blocks.11.attn.to_out.0.weight, pos_embed.proj.weight, transformer_blocks.0.norm1.linear.weight, transformer_blocks.16.ff.net.2.weight, transformer_blocks.2.ff.net.0.proj.bias, transformer_blocks.15.attn.add_v_proj.weight, transformer_blocks.16.norm1.linear.weight, transformer_blocks.22.attn.add_q_proj.bias, transformer_blocks.10.attn.add_v_proj.bias, transformer_blocks.13.attn.add_v_proj.weight, transformer_blocks.13.attn.to_v.weight, transformer_blocks.7.ff_context.net.0.proj.weight, transformer_blocks.0.norm1.linear.bias, transformer_blocks.11.ff.net.2.weight, transformer_blocks.20.attn.to_out.0.weight, transformer_bloc

✅ UNet is now on cuda:0


In [2]:
from peft import LoraConfig, get_peft_model

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["to_q","to_k","to_v"],
    lora_dropout=0.05,
    bias="none",
    # task_type omitted!
)
unet_lora = get_peft_model(unet, lora_cfg)


In [3]:
import torch

for name, mod in unet.named_modules():
    if "to_out" in name:
        print(name, type(mod))


down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out <class 'torch.nn.modules.container.ModuleList'>
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0 <class 'torch.nn.modules.linear.Linear'>
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.1 <class 'torch.nn.modules.dropout.Dropout'>
down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out <class 'torch.nn.modules.container.ModuleList'>
down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0 <class 'torch.nn.modules.linear.Linear'>
down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.1 <class 'torch.nn.modules.dropout.Dropout'>
down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_out <class 'torch.nn.modules.container.ModuleList'>
down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_out.0 <class 'torch.nn.modules.linear.Linear'>
down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_out.1 <class 'torch.nn.modules.dropout.Dropout'>
down_blocks.0.attentions.1.transformer_bloc

In [4]:
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from torch.utils.data import DataLoader
from transformers import CLIPTokenizer
from datasets import load_dataset
from torchvision import transforms
from PIL import Image

# ───── CONFIG ─────
MODEL_ID    = "stabilityai/stable-diffusion-3-medium-diffusers"
TRAIN_JSON  = "lora_dataset.json"
OUTPUT_DIR  = "lora_sd3_panel"
BATCH_SIZE  = 4
EPOCHS      = 3
LR          = 1e-4
WARMUP      = 0.03
MAX_LEN     = 64
SEED        = 42
# ───────────────────

torch.manual_seed(SEED)
accelerator = Accelerator(mixed_precision="fp16")
device = accelerator.device

# 1) Load SD3 pipeline just for VAE, scheduler & text_encoder
from diffusers import StableDiffusion3Pipeline
pipe = StableDiffusion3Pipeline.from_pretrained(
    MODEL_ID, torch_dtype=torch.float16, use_auth_token=True
).to(device)
vae            = pipe.vae
scheduler      = pipe.scheduler
text_encoder   = pipe.text_encoder
tokenizer      = CLIPTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer", use_auth_token=True)

# 2) Prepare DataLoader
img_tf = transforms.Compose([
    transforms.Resize((512,512), transforms.InterpolationMode.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])
ds = load_dataset("json", data_files=TRAIN_JSON)["train"]

def preprocess(ex):
    img = Image.open(ex["image"]).convert("RGB")
    ex["pixel_values"] = img_tf(img)
    ex["input_ids"]    = tokenizer(
        ex["caption"],
        padding="max_length", truncation=True, max_length=MAX_LEN,
        return_tensors="pt"
    ).input_ids[0]
    return ex

ds = ds.map(preprocess, remove_columns=["image","caption"])
ds.set_format(type="torch", columns=["pixel_values","input_ids"])

def collate_fn(batch):
    return {
        "pixel_values": torch.stack([b["pixel_values"] for b in batch]),
        "input_ids":    torch.stack([b["input_ids"]    for b in batch]),
    }

loader = DataLoader(
    ds, batch_size=BATCH_SIZE, shuffle=True,
    collate_fn=collate_fn, num_workers=4, pin_memory=True
)

# 3) Optimizer & LR scheduler on LoRA weights only
optimizer = torch.optim.AdamW(unet_lora.parameters(), lr=LR)
total_steps = len(loader) * EPOCHS
lr_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer, start_factor=WARMUP, total_iters=total_steps
)

# 4) Prepare for fp16 / multi‐GPU
unet_lora, optimizer, loader, lr_scheduler = accelerator.prepare(
    unet_lora, optimizer, loader, lr_scheduler
)

# 5) Training loop
for epoch in range(1, EPOCHS+1):
    unet_lora.train()
    total_loss = 0.0

    for batch in loader:
        # a) encode → latents
        pixel_values = batch["pixel_values"].to(device)
        input_ids    = batch["input_ids"].to(device)

        with torch.no_grad():
            latents = vae.encode(pixel_values).latent_dist.sample() * vae.config.scaling_factor

        # b) add noise
        noise      = torch.randn_like(latents)
        timesteps  = torch.randint(
            0, scheduler.config.num_train_timesteps,
            (latents.shape[0],), device=device
        ).long()
        noisy_latents = scheduler.add_noise(latents, noise, timesteps)

        # c) text → embeddings
        encoder_hidden_states = text_encoder(input_ids)[0]

        # d) UNet forward → predict noise
        noise_pred = unet_lora(
            noisy_latents,
            timesteps,
            encoder_hidden_states=encoder_hidden_states
        ).sample

        # e) loss & backward
        loss = F.mse_loss(noise_pred, noise)
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg = total_loss / len(loader)
    print(f"Epoch {epoch}/{EPOCHS} — avg loss: {avg:.4f}")

# 6) Save LoRA adapters
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
unet_lora.save_pretrained(OUTPUT_DIR)
print("✅ LoRA adapters saved to", OUTPUT_DIR)


Couldn't connect to the Hub: 401 Client Error. (Request ID: Root=1-6874cfc4-122701407c8d6fe7647b0473;3507708f-b01f-4f51-a614-7be3059f0cfa)

Repository Not Found for url: https://huggingface.co/api/models/stabilityai/stable-diffusion-3-medium-diffusers.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
User Access Token "solar2" is expired.
Will try to load from local cache.
Keyword arguments {'use_auth_token': True} are not expected by StableDiffusion3Pipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-6874cfc9-4fb3f7113274fa957c036788;ecd2e996-cb62-4eab-b373-0d5ac0ead6ad)

Repository Not Found for url: https://huggingface.co/api/models/stabilityai/stable-diffusion-3-medium-diffusers/tree/main/additional_chat_templates?recursive=False&expand=False.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
User Access Token "solar2" is expired